In [5]:
from my_module import Manager
# tk backend opens a new interactive window
%matplotlib tk
# run the calculations
manager = Manager("data/test.json")
manager.run()

AttributeError: 'Animation' object has no attribute 'start_animation'